In [1]:
import pandas as pd
from ekonlpy.tag import Mecab


In [ ]:
import os

In [ ]:
os.listdir(r'C:\Users\seon\Desktop\BOK_TEAM_5\03_Data')

In [ ]:
df1 = pd.read_csv('../03_Data/hankyung_금리_전처리.csv')
df2 = pd.read_csv('../03_Data/hankyung_통화정책_전처리.csv')
df3 = pd.read_csv('../03_Data/infomax_mp_preprcs.csv')
df4 = pd.read_csv('../03_Data/infomax_금리1_전처리.csv')
df5 = pd.read_csv('../03_Data/infomax_금리2_전처리.csv')
df6 = pd.read_csv('../03_Data/edaily_금리_전처리.csv')
df7 = pd.read_csv('../03_Data/edaily_통화정책_전처리.csv')

In [ ]:
df6 = df6.rename(columns={'contents':'content'})
df7 = df6.rename(columns={'contents':'content'})
df6 = df6[['date','title','content']]
df7 = df7[['date','title','content']]

In [ ]:
df1.drop_duplicates(subset='content',keep='first',inplace=True)
df2.drop_duplicates(subset='content',keep='first',inplace=True)
df3.drop_duplicates(subset='content',keep='first',inplace=True)
df4.drop_duplicates(subset='content',keep='first',inplace=True)
df5.drop_duplicates(subset='content',keep='first',inplace=True)
df6.drop_duplicates(subset='content',keep='first',inplace=True)
df7.drop_duplicates(subset='content',keep='first',inplace=True)

In [ ]:
df = pd.concat([df1,df2,df3,df4,df5,df6,df7],ignore_index=True)

In [ ]:
df = df.loc[:,['date','content']]

In [ ]:
df = df.groupby('date').sum().reset_index()

In [ ]:
df

In [ ]:
# uni-gram을 포함하기 위해서 mpck 기본 min_ngram설정 1로 바꿔주기!!(default값 2임)
def token_ngram(text):
    from ekonlpy.sentiment import MPCK
    mpck = MPCK()

    token = mpck.tokenize(text)
    ngram = mpck.ngramize(token)
        
    return ngram


In [ ]:
df['ngrams']= df['content'].apply(lambda x: token_ngram(str(x)))

In [ ]:
# df.to_csv('news_ngrams.csv',index=False,encoding='cp949')

In [2]:
df = pd.read_csv('./news_ngrams.csv',encoding='cp949')

In [3]:
df_rate = pd.read_csv('rate_labeling.csv')

In [4]:
new_df = pd.merge(df,df_rate,how='left')

In [5]:
new_df.loc[:,['date','content','ngrams','change']]

,date,content,ngrams,change
0,2011-01-01,이데일리 김춘동 기자 경제 전문가들은 새해 한국경제를 내다보면서 대체로 대 초반의 ...,"['과잉/NNG;유동성/NNG;경기/NNG;회복/NNG', '금리/NNG;인상/NN...",0
1,2011-01-02,증거가 아무리 많아도 미래를 정확히 예측할 수 없고 속는 사람만 비싼 대가를 치러야...,"['원자재/NNG;가격/NNG;상승/NNG;금리/NNG;인상/NNG', '유가/NN...",0
2,2011-01-03,신한은행은 세계 경기 회복 불확실성에 따라 국내 경기가 둔화할 것으로 예상하고 올해...,"['벤치마크/NNG;미/NNG;국채/NNG;금리/NNG;상승/NNG', '원자재/N...",0
3,2011-01-04,꾸준히 늘던 외국인 보유채권 잔액이 지난달 조원 넘게 감소해 궁금증을 자아내고 있다...,"['소비자/NNG;물가/NNG;상승률/NNG;물가/NNG;안정/NNG', '글로벌/...",0
4,2011-01-05,한경이코노미스트클럽 회원 가운데 이달 중 한국은행이 기준금리를 인상할 것이라고 예상...,"['소비자/NNG;물가/NNG;상승률/NNG;물가/NNG;안정/NNG', '원자재/...",0
...,...,...,...,...
4294,2022-10-08,수원 집값이 심상치 않습니다 가파른 금리 인상과 집값 고점 인식 확산으로 수원 뿐 ...,"['금리선물/NNG;시장/NNG;fed/NNG;금리/NNG;인상/NNG', 'fed...",0
4295,2022-10-09,한국은행이 오는 일 열리는 금융통화위원회 정례회의에서 기준금리를 포인트 올리는 빅스...,"['fed/NNG;매파/NNG;통화/NNG;긴축/NNG;선호/NNG', '금리/NN...",0
4296,2022-10-10,은행 예 적금 연 시대가 열리면서 예금과 적금 상품을 활용한 재테크에 투자자의 관심...,"['금리선물/NNG;시장/NNG;fed/NNG;금리/NNG;인상/NNG', '기업/...",0
4297,2022-10-11,세계 경기가 결국 침체의 늪을 피하지 못할 것이라는 비관론이 확산하고 있다 인플레이...,"['crs/NNG;irs/NNG;갭/NNG;스왑베이시스/NNG;축소/NNG', '금...",0


In [10]:
ngram_dict = {}
ngram_list = []

for ngram in new_df['ngrams']:
    for i in ngram[1:-1].split(','):
        ngram_list.append(i)
# ngram_unique = set(ngram_list)

# for i in ngram_unique:
#     ngram_dict[i] = {'hahawkish':0,'dovish':0}
# print(ngram_dict)

In [9]:
ngram_unique

{'',
 " '하락/NNG;주가/NNG;급등/NNG'",
 " '지수선물/NNG;내리/VV'",
 " '경기/NNG;불확실성/NNG;해소/NNG'",
 " '낮/VA;강세/NNG'",
 " '가계/NNG;대출/NNG;우려/NNG'",
 " '구제금융/NNG;부실/NNG'",
 " '채권시장/NNG;비둘기/NNG'",
 " '과열/NNG;인플레이션/NNG;압력/NNG'",
 " '인플레이션/NNG;회복/NNG'",
 " '경제/NNG;성장/NNG;불안/NNG'",
 " '기대/NNG;원자재/NNG;가격/NNG;상승/NNG'",
 " '높/VA;경기/NNG;상승/NNG'",
 " '부진/NNG;일시적/VAX;금리/NNG;인상/NNG'",
 " '부양책/NNG;철회/NNG'",
 " '소매/NNG;판매가격/NNG;예상/NNG;부진/NNG'",
 " '경기/NNG;판단/NNG;상향/NNG'",
 " 'ecb/NNG;금리/NNG;인상/NNG;긴축/NNG'",
 " '정책/NNG;신뢰/NNG;높/VV'",
 " '점차/MAG;저점/NNG;높/VV'",
 " '부동산/NNG;과열/NNG;억제/NNG'",
 " '회복/NNG;속도/NNG;상승/NNG'",
 " '금리/NNG;인하/NNG;기대/NNG;해소/NNG'",
 " '컨센서스/NNG;금리/NNG;상승/NNG'",
 " '신용위험/NNG;줄/VV'",
 " '안정/NNG;기대/NNG;높/VV'",
 " '대외/NNG;신인도/NNG;개선/NNG'",
 " '경기/NNG;회복/NNG;속도/NNG;높/VV'",
 " '위험/NNG;투자심리/NNG;강화/NNG'",
 " 'crs/NNG;상승/NNG;확대/NNG'",
 " '주택가격/NNG;불안/NNG'",
 " '높/VA;인플레이션/NNG;금리/NNG;인상/NNG'",
 " '외환보유액/NNG;줄/VV'",
 " '경기/NNG;부양책/NNG;재정/NNG;적자/NNG'",
 " '무역/NNG;증가/NNG'",
 " '소폭/MAG;상승/NNG;가능성/NNG;높/VA'",
 " 

In [7]:
ngram_dict

{'': {'hahawkish': 0, 'dovish': 0},
 " '하락/NNG;주가/NNG;급등/NNG'": {'hahawkish': 0, 'dovish': 0},
 " '지수선물/NNG;내리/VV'": {'hahawkish': 0, 'dovish': 0},
 " '경기/NNG;불확실성/NNG;해소/NNG'": {'hahawkish': 0, 'dovish': 0},
 " '낮/VA;강세/NNG'": {'hahawkish': 0, 'dovish': 0},
 " '가계/NNG;대출/NNG;우려/NNG'": {'hahawkish': 0, 'dovish': 0},
 " '구제금융/NNG;부실/NNG'": {'hahawkish': 0, 'dovish': 0},
 " '채권시장/NNG;비둘기/NNG'": {'hahawkish': 0, 'dovish': 0},
 " '과열/NNG;인플레이션/NNG;압력/NNG'": {'hahawkish': 0, 'dovish': 0},
 " '인플레이션/NNG;회복/NNG'": {'hahawkish': 0, 'dovish': 0},
 " '경제/NNG;성장/NNG;불안/NNG'": {'hahawkish': 0, 'dovish': 0},
 " '기대/NNG;원자재/NNG;가격/NNG;상승/NNG'": {'hahawkish': 0, 'dovish': 0},
 " '높/VA;경기/NNG;상승/NNG'": {'hahawkish': 0, 'dovish': 0},
 " '부진/NNG;일시적/VAX;금리/NNG;인상/NNG'": {'hahawkish': 0, 'dovish': 0},
 " '부양책/NNG;철회/NNG'": {'hahawkish': 0, 'dovish': 0},
 " '소매/NNG;판매가격/NNG;예상/NNG;부진/NNG'": {'hahawkish': 0, 'dovish': 0},
 " '경기/NNG;판단/NNG;상향/NNG'": {'hahawkish': 0, 'dovish': 0},
 " 'ecb/NNG;금리/NNG;인상/NNG;